In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.datasets import Amazon
from torch_geometric.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# 데이터셋 불러오기
dataset = Amazon(root="./tmp/", name="Computers")
data = dataset[0]

In [ ]:
data

In [ ]:
# 데이터를 training과 test로 분리
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = 1  # 80%의 노드를 학습에 사용
test_mask = ~train_mask

In [ ]:
# GATv2 모델 정의
class GATv2Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GATv2Net, self).__init__()
        self.conv1 = GATv2Conv(in_channels, 128, heads=4)
        self.bn1 = torch.nn.BatchNorm1d(128 * 4)
        self.conv2 = GATv2Conv(128 * 4, out_channels, heads=1, concat=False)
        self.bn2 = torch.nn.BatchNorm1d(out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# 모델과 옵티마이저 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATv2Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=5e-4)

# Learning Rate 스케쥴러
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Early Stopping 파라미터
patience = 20
best_loss = None
epochs_no_improve = 0

In [ ]:
# 학습 루프
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    scheduler.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    # Early Stopping
    if best_loss is None:
        best_loss = loss.item()
    elif best_loss > loss.item():
        best_loss = loss.item()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print("Early stopping!")
            break

In [ ]:
# 모델을 평가 모드로 설정
model.eval()
_, pred = model(data).max(dim=1)

# 테스트 데이터에서의 예측과 실제 라벨
pred_test = pred[test_mask].cpu().numpy()
y_test = data.y[test_mask].cpu().numpy()

# 정확도 계산
correct = pred[test_mask].eq(data.y[test_mask]).sum().item()
accuracy = correct / test_mask.sum().item()

# 정밀도, 재현율, F1 점수 계산
precision = precision_score(y_test, pred_test, average='macro')
recall = recall_score(y_test, pred_test, average='macro')
f1 = f1_score(y_test, pred_test, average='macro')

print(f"Test accuracy: {accuracy:.4f}")
print(f"Test precision: {precision:.4f}")
print(f"Test recall: {recall:.4f}")
print(f"Test F1 score: {f1:.4f}")